# Model Training

CELL 1 - Import Libraries

In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import mysql.connector

CELL 2 - Connect to Database

In [3]:
def get_connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="Rs@969383",
        database="pollution_db"
    )

conn = get_connection()

query = """
SELECT record_date, AVG(aqi) as avg_aqi
FROM aqi_readings ar
JOIN locations l ON ar.location_id = l.location_id
GROUP BY record_date
ORDER BY record_date
"""

df = pd.read_sql(query, conn)
conn.close()

df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_12536\4205288876.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,record_date,avg_aqi
0,2026-02-01,250.0000
1,2026-02-10,227.7778
2,2026-02-20,107.0000


CELL 3 - Prepare Dataset

In [4]:
df["record_date"] = pd.to_datetime(df["record_date"])

df["days"] = (df["record_date"] - df["record_date"].min()).dt.days

X = df[["days"]]
y = df["avg_aqi"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

CELL 4 - Train Model

In [5]:
model = LinearRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

print("MAE:", mean_absolute_error(y_test, predictions))

MAE: 96.0864666666667


CELL 5 - Save Model

In [6]:
joblib.dump(model, "aqi_model.pkl")
print("Model saved successfully!")

Model saved successfully!
